In [12]:
import csv
import sys
import re
import matplotlib.pyplot as plt
import numpy as np

# Cutoff for rare variants 

In [13]:
cutoff  = 0.000001

In [14]:
def writelist (mylist, myfile): 
    fp =  open(myfile,'a+')
    for idx,item in enumerate(mylist):
            fp.write(item)
            fp.write('\t')
    fp.write('\n')
    fp.close()

In [15]:
def intersections(list1, list2):
    intersection_list = []
    for i in list1: 
        if i in list2: 
                intersection_list.append(i)
    return intersection_list

In [16]:
def difference(list1, list2):
    unique_list = []
    for i in list1: 
        if i not in list2: 
                unique_list.append(i)
    return unique_list

In [17]:
def exist (list1, list2): 
    found_list = []
    for i in list1: 
            if i not in found_list: 
                    found_list.append(i)
    for i in list2: 
            if i not in found_list: 
                    found_list.append(i)
    return found_list 

In [18]:
def common_variants(variants1, variants2, outfile): 
        fp = open(outfile, 'w')
        for i in variants1: 
               for j in variants2: 
                     count = 0 
                     for a in range(0,4): 
                            if i[a] == j[a]:
                                count +=1
                     if count == 4: 
                          for item in (i):
                                fp.write(item)
                                fp.write('\t')
                          fp.write('\n')

# Read annovar annotation file

In [19]:
-

In [20]:
rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics, muttype = load_annovar('F366M1.hg19_multianno.txt')
count = 0
print ('Starting ....')
outfile = 'f366m1.raremaf.filtered.txt'
f366m1_raremaf_genes = []
f366m1_raremaf_variants = [] 
print (cutoff)
for i in range(0,len(exacall) ): 
    if ( (gnomall[i] != '.') and   (exacall[i] !='.' ) ) :
        if (  (float(gnomall[i]) < cutoff) and (float(exacall[i]) < cutoff)   and (functions[i] =='exonic') and ( muttype[i] != 'synonymous SNV' )): 
            count += 1 
            writelist(rows[i], outfile)
            print(rs[i], gene[i], exacall[i],gnomall[i], muttype[i]) 
            if gene[i] not in f366m1_raremaf_genes: 
                    f366m1_raremaf_genes.append(gene[i])
            f366m1_raremaf_variants.append(rows[i])
            #var = rs[i] + '.' + gene[i] +'.' +exacall[i] +'.'+ gnomall[i]        
            #if var not in f366m1_raremaf_variants: 
                #f366m1_raremaf_variants.append(var)
    elif ( (gnomall[i]=='.') and (exacall[i].isdigit())):
           print(gnomall[i])
           if ((float(exacall[i]) < cutoff)and (functions[i] =='exonic')  and ( muttype[i] != 'synonymous SNV' )): 
                count += 1 
                writelist(rows[i], outfile)
                print(rs[i], gene[i], exacall[i],gnomall[i], muttype[i])
                if gene[i] not in f366m1_raremaf_genes: 
                        f366m1_raremaf_genes.append(gene[i])
                f366m1_raremaf_variants.append(rows[i])
                #var = rs[i] + '.' + gene[i] +'.' +exacall[i] +'.'+ gnomall[i]
                #if var not in f366m1_raremaf_variants: 
                    #f366m1_raremaf_variants.append(var)
           
    elif (( exacall[i]=='.') and   (gnomall[i].isdigit())): 
            print(exacall[i])
            if( (float (gnomall[i]) < cutoff)  and (functions[i] =='exonic') and  ( muttype[i] != 'synonymous SNV' )): 
                count += 1 
                writelist(rows[i], outfile)
                print(rs[i], gene[i], exacall[i],gnomall[i], muttype[i])
                if gene[i] not in  f366m1_raremaf_genes: 
                         f366m1_raremaf_genes.append(gene[i])
                f366m1_raremaf_variants.append(rows[i])       
                #var = rs[i] + '.' + gene[i] +'.' +exacall[i] +'.'+ gnomall[i]
                #if var not in f366m1_raremaf_variants: 
                    #f366m1_raremaf_variants.append(var)
print ('We reached printing stage')
print ('Number of variants with gnomall and/or exac <', cutoff, 'is:',count)
print (f366m1_raremaf_variants)
print('========================================================')
print(f366m1_raremaf_genes)
print('Done')

ExonicFunc.refGene
ExAC_ALL
Examined_samples
gnomAD_exome_ALL
Gene.refGene
Func.refGene
MetaLR_pred
Starting ....
1e-06
.
.
.
.
.
.
('.', 'TEKT4', '.', '0', 'nonsynonymous SNV')
.
('.', 'TEKT4', '.', '0', 'nonsynonymous SNV')
.
.
.
.
('.', 'ANKRD36B', '.', '0', '.')
.
('.', 'ANKRD36B', '.', '0', '.')
.
('.', 'ANKRD36B', '.', '0', '.')
('.', 'STK39', '0', '0', 'nonframeshift insertion')
.
.
.
('.', 'PLCL2', '0', '.', '.')
.
.
.
.
.
.
.
.
.
('.', 'FRG2C', '0', '.', 'nonsynonymous SNV')
.
.
.
('.', 'ZNF717', '.', '0', 'nonsynonymous SNV')
.
('.', 'ZNF717', '.', '0', 'nonsynonymous SNV')
.
('.', 'ZNF717', '.', '0', 'nonsynonymous SNV')
.
.
.
('.', 'RYK', '0', '.', 'frameshift insertion')
.
.
.
.
.
.
.
('.', 'SDK1', '0', '.', 'nonsynonymous SNV')
.
('.', 'VWDE', '.', '0', 'nonsynonymous SNV')
.
('.', 'KBTBD11', '.', '0', 'nonsynonymous SNV')
.
.
.
('.', 'ANKRD20A4', '.', '0', 'nonsynonymous SNV')
.
('.', 'ANKRD20A4', '.', '0', 'nonsynonymous SNV')
.
('.', 'ADRB1', '0', '0', 'nonsynonymous S

In [21]:
-

ExonicFunc.refGene
ExAC_ALL
Examined_samples
gnomAD_exome_ALL
Gene.refGene
Func.refGene
MetaLR_pred
Starting ....
1e-06
.
.
.
.
.
.
.
.
('rs200427142', 'IGFN1', '.', '0', 'nonsynonymous SNV')
.
('rs376333361', 'TEKT4', '.', '0', 'nonsynonymous SNV')
.
('rs142133592', 'TEKT4', '.', '0', 'nonsynonymous SNV')
.
('rs78179792', 'ANKRD36C', '.', '0', 'nonsynonymous SNV')
.
.
.
.
.
('rs372700939', 'IGFBP2', '.', '0', 'nonsynonymous SNV')
.
('rs377689849', 'IGFBP2', '.', '0', 'nonsynonymous SNV')
.
('rs185489391', 'IGFBP2', '.', '0', 'nonsynonymous SNV')
.
.
.
('rs71624922', 'SYN2', '0', '.', '.')
.
.
.
('rs368284573', 'PLCL2', '0', '.', '.')
.
.
.
.
.
.
.
('rs199810415', 'FRG2C', '0', '.', 'nonsynonymous SNV')
.
.
.
.
.
.
('rs148249548', 'ZNF717', '.', '0', 'nonsynonymous SNV')
.
.
('rs149887072', 'ZNF717', '.', '0', 'nonsynonymous SNV')
.
('rs112847134', 'ZNF717', '.', '0', 'nonsynonymous SNV')
.
('rs74861398', 'ZNF717', '.', '0', 'nonsynonymous SNV')
.
.
.
.
('.', 'OR5H14', '.', '0', 'nonsy

In [22]:
f366_m1_m2 = intersections(f366m1_raremaf_genes,f366m2_raremaf_genes) 
print (f366_m1_m2)

['TEKT4', 'PLCL2', 'FRG2C', 'ZNF717', 'KBTBD11', 'BNC1', 'HYDIN', 'CDC27', 'KIR3DL1', 'KIR2DS4']
